# DrawbridgeからのデータでIDFAデータをテーブル化

In [64]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

from ailab_tools.utils import split_and_stack

In [65]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190610


味見　中身の確認

In [51]:
#ファイル読込
df = pd.read_csv("/home/yusuke_akada/notebook/adhoc/device/part-00000_after.txt", header=None , names=['column'])

In [52]:
#ファイル読込
df = pd.read_csv("/home/yusuke_akada/notebook/adhoc/device/part-00000_after.txt", header=None , names=['column'])

#カラム分割し、ユニークなDrawbridge_consumer_idとidfaのidなどを分離
splitted = df['column'].str.split('[|]', 1, expand=True)
splitted = splitted.rename(columns={0: 'drawbridge_consumer_id', 1:'uid'})

#Drawbridge_consumer_id単位でとidfaなどのidを縦持ち化
stacked = split_and_stack(splitted, "uid", sep="|")
stacked['drawbridge_consumer_id'] = stacked['drawbridge_consumer_id'].str.strip('drawbridge_consumer_id:')
stacked["Split"] = stacked["uid"].str.split(":")
stacked["type"] = stacked["Split"].str.get(0)
stacked["uid"] = stacked["Split"].str.get(1)
stacked= stacked.drop(['Split'], axis=1)
stacked.head()

,drawbridge_consumer_id,uid,type
0,001d5e82286be61794e2a46662107018,1407000232427482,cookie
1,001d5e82286be61794e2a46662107018,1405000199612114,cookie
2,001d5e82286be61794e2a46662107018,72FF704B-72EE-492C-92F4-4B9D65367CED,android_idfa
3,001d5e82286be61794e2a46662107018,75C46B5F-7543-404E-83EA-495162C28C96,android_idfa
4,002ccb97b4583053fd158374241cab38,1462000004321097,cookie


In [ ]:
df_tmp = stacked[(stacked['type'] == "cookie")]
df_tmp = df_tmp.astype({'uid':int})
df_tmp = df_tmp.rename(columns={'uid':'smn_uid'})
df_tmp.head()

In [25]:
print(df_tmp.dtypes)

drawbridge_consumer_id    object
uid                        int64
type                      object
dtype: object


In [42]:
df_tmp = stacked[(stacked['type'] == "android_idfa")]
df_tmp['uid'] = df_tmp['uid'].str.replace('-', '')
df_tmp = df_tmp.rename(columns={'uid':'advertising_id'})
df_tmp.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,drawbridge_consumer_id,advertising_id,type
2,001d5e82286be61794e2a46662107018,72FF704B72EE492C92F44B9D65367CED,android_idfa
3,001d5e82286be61794e2a46662107018,75C46B5F7543404E83EA495162C28C96,android_idfa
91,03b9276131dab1deab414124d90565c1,8920D024000F418A9A3CCC8D391B4EC1,android_idfa
92,03b9276131dab1deab414124d90565c1,998B6FAAD92B46ED8AB75A61A2FD69AE,android_idfa
93,03b9276131dab1deab414124d90565c1,A1CC4AAA29E34E02A4EFF325CB8BCBE3,android_idfa


stacked.groupby(["type"])["uid"].count()

print(stacked.dtypes)

分割されたテキストファイルを縦持ち＆統合しImpalaにUp<br>
smn_uidだけのファイルとadvertising_idだけの2種類を最終的には作る

まずはsmn_uid

In [46]:
%%slack_notify @yusuke_akada
uid_table = "segment.drawbridge_smn_uids"   # Drawbridge_idとidfa_idの最終的な対応table
temp_table = f"dm_tmp.ya_temp_drawbridge_smn_uids_{today_yymmdd}"    # 分割処理用のtemporary table 実行後dropされる


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:

    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            drawbridge_consumer_id string
            ,smn_uid bigint
            ,type string
            )
        
        partitioned by 
            (
            block int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query)    

    # テキストファイル単体ごとに読込、DataFrame化
    for i in range(0,200):
            if i < 10:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/adhoc/device/part-0000{i}_after.txt", header=None, names=['column'])
            elif 10 <= i < 100:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/adhoc/device/part-000{i}_after.txt", header=None, names=['column'])
            else:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/adhoc/device/part-00{i}_after.txt", header=None, names=['column'])
                
            splitted = df['column'].str.split('[|]', 1, expand=True)
            splitted = splitted.rename(columns={0: 'drawbridge_consumer_id', 1:'uid'})
            
            #Drawbridge_consumer_id単位でとidfaなどのidを縦持ち化
            stacked = split_and_stack(splitted, "uid", sep="|")
            stacked['drawbridge_consumer_id'] = stacked['drawbridge_consumer_id'].str.strip('drawbridge_consumer_id:')
            stacked["Split"] = stacked["uid"].str.split(":")
            stacked["type"] = stacked["Split"].str.get(0)
            stacked["uid"] = stacked["Split"].str.get(1)
            stacked= stacked.drop(['Split'], axis=1)
            
            stacked = stacked[(stacked['type'] == "cookie")]
            stacked = stacked.astype({'uid':int})
            stacked = stacked.rename(columns={'uid':'smn_uid'})
        
        # ImpalaにUP
            ir.upload_dataframe(temp_table, stacked, ['string','bigint','string'])
        
        # 親テーブルにInsert
            query = (
                f"""
                insert overwrite table {uid_table}
                    (
                    drawbridge_consumer_id
                    ,smn_uid
                    ,type 
                    )

                partition
                    (
                    block = {i}
                    )

                select
                 drawbridge_consumer_id
                 ,smn_uid
                 ,type
                from {temp_table}
                """
            )
            ir.execute_query_no_ret(query) 

            # tempテーブルdrop
            ir.drop_tables([temp_table])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

advertising_idで

    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            drawbridge_consumer_id string
            ,advertising_id string
            ,type string
            )
        
        partitioned by 
            (
            block int
            )
        
        stored as parquet
        """
    )
    ir.execute_query_no_ret(query)    

In [67]:
%%slack_notify @yusuke_akada
uid_table = "segment.drawbridge_idfas"   # Drawbridge_idとidfa_idの最終的な対応table
temp_table = f"dm_tmp.ya_temp_drawbridge_idfas_{today_yymmdd}"    # 分割処理用のtemporary table 実行後dropされる


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:


    
    
    


    # テキストファイル単体ごとに読込、DataFrame化
    for i in range(0,200):
            if i < 10:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/adhoc/device/part-0000{i}_after.txt", header=None, names=['column'])
            elif 10 <= i < 100:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/adhoc/device/part-000{i}_after.txt", header=None, names=['column'])
            else:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/adhoc/device/part-00{i}_after.txt", header=None, names=['column'])
                
            splitted = df['column'].str.split('[|]', 1, expand=True)
            splitted = splitted.rename(columns={0: 'drawbridge_consumer_id', 1:'uid'})
            
            #Drawbridge_consumer_id単位でとidfaなどのidを縦持ち化
            stacked = split_and_stack(splitted, "uid", sep="|")
            stacked['drawbridge_consumer_id'] = stacked['drawbridge_consumer_id'].str.strip('drawbridge_consumer_id:')
            stacked["Split"] = stacked["uid"].str.split(":")
            stacked["type"] = stacked["Split"].str.get(0)
            stacked["uid"] = stacked["Split"].str.get(1)
            stacked= stacked.drop(['Split'], axis=1)
            
            stacked = stacked[(stacked['type'] == "android_idfa")|(stacked['type'] == "ios_idfa")]
            stacked['uid'] = stacked['uid'].str.replace('-', '')
            stacked = stacked.rename(columns={'uid':'advertising_id'})
        
        # ImpalaにUP
            ir.upload_dataframe(temp_table, stacked, ['string','string','string'])
        
        # 親テーブルにInsert
            query = (
                f"""
                insert overwrite table {uid_table}
                    (
                    drawbridge_consumer_id
                    ,advertising_id
                    ,type 
                    )

                partition
                    (
                    block = {i}
                    )

                select
                 drawbridge_consumer_id
                 ,advertising_id
                 ,type
                from {temp_table}
                """
            )
            ir.execute_query_no_ret(query) 

            # tempテーブルdrop
            ir.drop_tables([temp_table])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:Create empty 